In [1]:
def get_datatable(startDate,endDate,isoregion):
    """
    Web scarpping the data table from power calendar
    Args:
        startDate:string type, e.g '2018-2-1'
        endDate:string type e.g '2019-12-20'
        isoregion: string type, e.g. 'ERCOT'
    Return a pandas dataframe
    """
    import pandas as pd
    import numpy as np
    east = ['PJM','MISO','ERCOT','SPP','NYISO']
    west = ['WECC','CAISO']
    if isoregion in east:
        region = 'ERCOT'
    elif isoregion in west:
        region = 'CAISO'
    else:
        raise ValueError('Incorrect ISO region name')
    url = 'https://www.energygps.com/HomeTools/PowerCalendar?isoregion='+region+'&scale=m&aname=xl&startDate='+startDate+'&endDate='+endDate
    df = pd.read_table(url)
    #drop the columns with 'DISCLAIMER' info
    df = df.drop(columns=['DISCLAIMER: ENERGY GPS HAS USED SOURCES AND METHODS THAT IT BELIEVES TO BE RELIABLE; HOWEVER, ENERGY GPS DOES NOT WARRANT THE CORRECTNESS, COMPLETENESS, CURRENTNESS, OR ACCURACY OF THE INFORMATION. IN NO EVENT SHALL ENERGY GPS BE LIABLE FOR ANY SPECIAL, INDIRECT, INCIDENTAL, CONSEQUENTIAL, OR PUNITIVE DAMAGES (INCLUDING WITHOUT LIMITATION DAMAGES FOR LOSS OF PROFITS, BUSINESS INTERRUPTION, LOSS OF DATA, OR LOSS OF BUSINESS INFORMATION), WHETHER BASED ON CONTRACT, TORT, OR OTHER LEGAL THEORY, IN CONNECTION WITH THIS REPORT OR THE USE OF OR INABILITY TO USE THE INFORMATION AND ANALYSIS CONTAINED IN IT.'])
    #drop the region level index
    df.index = df.index.droplevel(level=0)
    #reset df index 
    df = df.reset_index()
    #reset df columns names
    df.columns = df.loc[0].values
    df = df[1:]
    df.rename(columns = {'Peak':'onpeak'}, inplace = True)
    df.set_index(df['StartDate'],inplace=True)
    df = df.rename(str.lower, axis='columns')
    #set hours data to float type
    for column in df.columns[2:11]:
        df[column] = df[column].astype(float)
    return df

In [2]:
def get_hours(isoregion,peaktype,period):
    """
    calculate hours during the given date
    Args: 
       iso: a character, one of PJM, MISO, ERCOT, SPP, NYISO, WECC, CAISO
       peak.type: a character, one of onpeak, offpeak, flat, 2*16H, 7*8
       period: a character 4 types, daily, monthly, quarterly, annnually
    Return a list:
       isoregion
       peaktype
       startDate
       endDate
       numHour
    """
    from datetime import datetime
    import calendar
    import holidays
    from datetime import date,timedelta
    us_holidays = holidays.US()
    #calculate the yearly hours in different peak types and different iso region 
    if period[:4] =='Year':
        startDate = date(int('20'+period[4:]),1,1)
        endDate = date(int('20'+period[4:]),12,31)
        daygenerator = [startDate + timedelta(x) for x in range((endDate - startDate).days+1)]
        h_78 = sum(8 for day in daygenerator if day.weekday() < 7)
        df = get_datatable('20'+period[4:]+'-1-1','20'+period[4:]+'-12-31',isoregion)
        flat,onpeak,offpeak = [df[columns].sum() for columns in ['flat','onpeak','offpeak']]
        if isoregion in ['CAISO','WECC']:
            h_216 = sum(16 for day in daygenerator if ((day.weekday()==0)&(day in us_holidays))|(day.weekday()==6))
        else:
            h_216 = sum(16 for day in daygenerator if ((day.weekday()==0)&(day in us_holidays))|(day.weekday()>=5))
    #calculate the quarterly hours in different peak types and different iso region 
    elif period[4] =='Q':
        quarter = int(period[5])
        df = get_datatable(period[:4]+'-1-1',period[:4]+'-12-31',isoregion)
        flat,onpeak,offpeak = [df[df['quarter']==quarter][columns].sum() for columns in ['flat','onpeak','offpeak']]
        from datetime import datetime
        startDate = datetime.strptime(df[df['quarter']==quarter]["startdate"][0], '%Y-%m-%d')
        endDate= datetime.strptime(df[df['quarter']==quarter]["enddate"][-1], '%Y-%m-%d')
        daygenerator = [startDate + timedelta(x) for x in range((endDate - startDate).days+1)]
        h_78 = sum(8 for day in daygenerator if day.weekday() < 7)
        if isoregion in ['CAISO','WECC']:
            h_216 = sum(16 for day in daygenerator if ((day.weekday()==0)&(day in us_holidays))|(day.weekday()==6))
        else:
            h_216 = sum(16 for day in daygenerator if ((day.weekday()==0)&(day in us_holidays))|(day.weekday()>=5))
    #calculate the monthly hours in different peak types and different iso region (MISO's hours changes when enter daylight saving)
    elif len(period)==5:
        month = datetime.strptime(period[:3], '%b').month
        year = int('20' + period[3:])
        import calendar
        lastday = calendar.monthrange(year, month)[1]
        df = get_datatable(str(year)+'-'+str(month)+'-1',str(year)+'-'+str(month)+'-'+str(lastday),isoregion)
        if (isoregion == 'MISO')&(year<=2007)&(4<=month<=10):
            flat,onpeak,offpeak = [df[columns].sum() for columns in ['flat','onpeak','offpeak']]+1
        elif (isoregion == 'MISO')&(year>2007)&(3<=month<=11):
            flat,onpeak,offpeak = [df[columns].sum() for columns in ['flat','onpeak','offpeak']]+1
        else:
            startDate = date(year,month,1)
            endDate = date(year,month,lastday)
            daygenerator = [startDate + timedelta(x) for x in range((endDate - startDate).days+1)]
            flat,onpeak,offpeak = [df[columns].sum() for columns in ['flat','onpeak','offpeak']]
            h_78 = sum(8 for day in daygenerator if day.weekday() < 7)
            if isoregion in ['CAISO','WECC']:
                h_216 = sum(16 for day in daygenerator if ((day.weekday()==0)&(day in us_holidays))|(day.weekday()==6))
            else:
                h_216 = sum(16 for day in daygenerator if ((day.weekday()==0)&(day in us_holidays))|(day.weekday()>=5))
    #calculate the daily hours in different peak types and different iso region
    elif len(period)>=8:
        startDate,endDate = period,period
        df = get_datatable(startDate,endDate,isoregion)
        flat,onpeak,offpeak = [df[columns].sum() for columns in ['flat','onpeak','offpeak']]
        h_78 = 8
        h_216 = df['1x16h']
    #Translate the peaktype into hours data
    if peaktype == 'flat':
        hours = flat
    elif peaktype == 'onpeak':
        hours = onpeak;
    elif peaktype == 'offpeak':
        hours = offpeak
    elif peaktype == '7*8':
        hours = h_78
    else:
        hours = h_216    
    return [isoregion,peaktype,str(startDate),str(endDate),int(hours)]

In [3]:
#let's test the function!
isoregion,peaktype,startDate,endDate,hours = get_hours('ERCOT','onpeak','May19')
print('ISO:',isoregion,'\nPeak Type:',peaktype,'\nStart Date:',startDate,'\nEnd Date',endDate,'\nHours',hours)

ISO: ERCOT 
Peak Type: onpeak 
Start Date: 2019-05-01 
End Date 2019-05-31 
Hours 352


In [4]:
isoregion,peaktype,startDate,endDate,hours = get_hours('WECC','7*8','2019-4-22')
print('ISO:',isoregion,'\nPeak Type:',peaktype,'\nStart Date:',startDate,'\nEnd Date',endDate,'\nHours',hours)

ISO: WECC 
Peak Type: 7*8 
Start Date: 2019-4-22 
End Date 2019-4-22 
Hours 8


In [5]:
isoregion,peaktype,startDate,endDate,hours = get_hours('CAISO','flat','Year18')
print('ISO:',isoregion,'\nPeak Type:',peaktype,'\nStart Date:',startDate,'\nEnd Date',endDate,'\nHours',hours)

ISO: CAISO 
Peak Type: flat 
Start Date: 2018-01-01 
End Date 2018-12-31 
Hours 8760


In [6]:
isoregion,peaktype,startDate,endDate,hours = get_hours('SPP','offpeak','2019Q1')
print('ISO:',isoregion,'\nPeak Type:',peaktype,'\nStart Date:',startDate,'\nEnd Date',endDate,'\nHours',hours)

ISO: SPP 
Peak Type: offpeak 
Start Date: 2019-01-01 00:00:00 
End Date 2019-03-31 00:00:00 
Hours 1151
